## Group 1: Week 12

# 'Game of Thrones' Story Through a Data Science Narrative

****

In [1]:
# import required libraries
import os
import csv
import pandas as pd

### Load Data

In [2]:
# set the path to the data files directory
home = os.getcwd()
the_path = home + '/Data/'

# read the csv files into dataframes
data_list = ['battles', 'deaths', 'predictions']

for data in data_list:
    file_path = the_path + str(data) + '.csv'
    the_code = str(data) + ' = pd.read_csv(file_path)'
    exec(the_code)

predictions.head()

,S.No,actual,pred,alive,plod,name,title,male,culture,dateOfBirth,...,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity,isAlive
0,1,0,0,0.054,0.946,Viserys II Targaryen,NaN,1,NaN,NaN,...,0.0,NaN,0,0,NaN,11,1,1,0.605351,0
1,2,1,0,0.387,0.613,Walder Frey,Lord of the Crossing,1,Rivermen,208.0,...,NaN,1.0,1,1,97.0,1,1,1,0.896321,1
2,3,1,0,0.493,0.507,Addison Hill,Ser,1,NaN,NaN,...,NaN,NaN,0,1,NaN,0,0,0,0.267559,1
3,4,0,0,0.076,0.924,Aemma Arryn,Queen,0,NaN,82.0,...,NaN,0.0,1,1,23.0,0,0,0,0.183946,0
4,5,1,1,0.617,0.383,Sylva Santagar,Greenstone,0,Dornish,276.0,...,NaN,1.0,1,1,29.0,0,0,0,0.043478,1
